# this project is finished by Tianlin Zhang(A20345584) and Li ma(A20346352)

In [38]:
import os
import subprocess
import pandas as pd
import numpy as np
from __future__ import print_function
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_curve  
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV,Ridge
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import Lars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

### 0. preparation 
before using the different training model, we should deal with the data first, in this part we will create the trainning set of clean and full data, as well as the label for regression and classification

 a. let's deal with the clean data first:

In [141]:
#read the clean data using pandas
df = pd.read_csv("communities-crime-clean.csv")
#define a function for classification output label 
def highcrime (row):
    if (row['ViolentCrimesPerPop'] > 0.1):
        return 1
    return 0
tmp = df.apply(lambda row:highcrime(row),axis = 1)
#we get the y for classification and y for regression
clean_clf_y = tmp.values.tolist()
clean_reg_y = df['ViolentCrimesPerPop']
#now del the non predictive feature
del df["state"]
del df["communityname"]
del df["fold"]
del df['ViolentCrimesPerPop']
# now we get the clean training set x
clean_X = df
#and the name of all the features
clean_features = list(df.columns[:])

b. now deal with the full data:

In [320]:
#read the full data using pandas
df2 = pd.read_csv("communities-crime-full.csv")
#deal with the '?' and unralated feature
df2 = df2.replace('?', np.nan)
del df2["state"]
del df2["county"]
del df2["community"]
del df2["communityname"]
del df2["fold"]
#we get the y for regression
full_reg_y = df2["ViolentCrimesPerPop"]
del df2["ViolentCrimesPerPop"]
#deal with it to get the clf y
full_clf_y = full_reg_y.copy()
for index, item in enumerate(full_clf_y):
    if (item > 0.1):
        full_clf_y[index] = 1
    else:
        full_clf_y[index] = 0
#we deal the x now and fill the missing data using mean of the exist data
imp = Imputer(missing_values= "NaN", strategy='mean', axis=0)
full_X = imp.fit_transform(df2)
#and all the features
full_features = list(df2.columns[:])

now we can do the project now
### 1. Decision Tree

#### a. get the crime rate and calculate the percentage of positive and negative example

In [4]:
#the highCrime is the same as clean_clf_y
highCrime = clean_clf_y
#the positive number and neg number is 
pos = highCrime.count(1)
neg = highCrime.count(0)
#the percentage
print("the positve percentage is:")
print(pos/len(highCrime))
print("the negative percentage is:")
print(neg/len(highCrime))

the positve percentage is:
0.6271951831409934
the negative percentage is:
0.3728048168590065


#### b.  Use DecisionTreeClassifier to learn a decision tree to predict highCrime

In [5]:
#init the DT clf
dt_clf = DecisionTreeClassifier()
#fit the data
dt_clf.fit(clean_X,clean_clf_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

 i. the accuracy, precision and recall are as follows:

In [6]:
answer = dt_clf.predict_proba(clean_X)[:,1]  
accuracy = accuracy_score(clean_clf_y, dt_clf.predict(clean_X))  
print("the accuracy is ",accuracy)
print(classification_report(clean_clf_y, answer, target_names = ['negative', 'positive']))

the accuracy is  1.0
             precision    recall  f1-score   support

   negative       1.00      1.00      1.00       743
   positive       1.00      1.00      1.00      1250

avg / total       1.00      1.00      1.00      1993



ii. now let's find the main features according to the gini importance

In [7]:
main_feature = dict(zip(clean_features,dt_clf.feature_importances_))
for index,w in enumerate(sorted(main_feature, key=main_feature.get, reverse=True)):
    print(w ,main_feature[w])
    if index == 10:
        break

PctKids2Par 0.362097595066
racePctWhite 0.0888168559837
racePctHisp 0.0485853556595
PctLess9thGrade 0.0235240057788
PctEmplManu 0.0181445204434
MedRent 0.0164670739976
PctWOFullPlumb 0.01450624349
PctVacMore6Mos 0.0128805049573
PctEmploy 0.0121760645547
PctRecImmig5 0.0115733067422
PctSameState85 0.0115333819062


The above result is make sense, because the decision tree is create by get the minimum entropy at every decision, and from the data we can find that if we order the data with the main features above, they are all high related to the crime rate. So the above result make sense.

#### c. Now apply cross-validation (cross_val_score) to do 10-fold cross-validation to estimate the out-of-training accuracy of decision tree learning for this task.

i.	What are the 10-fold cross-validation accuracy, precision, and recall?

In [145]:
ten_fold_accuracy = cross_val_score(dt_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
ten_fold_precision = cross_val_score(dt_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
ten_fold_recall = cross_val_score(dt_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(ten_fold_precision)
print("the recall for 10-fold cv is:")
print(ten_fold_recall)
print("the avg accuracy is:",sum(ten_fold_accuracy)/10)
print("the avg precision is:",sum(ten_fold_precision)/10)
print("the avg recall is:",sum(ten_fold_recall)/10)

the accuracy for 10-fold cv is:
[ 0.725       0.815       0.78        0.76884422  0.69849246  0.65829146
  0.74371859  0.69346734  0.76884422  0.71356784]
the precision for 10-fold cv is:
[ 0.75        0.81203008  0.77692308  0.76865672  0.76530612  0.76923077
  0.75182482  0.83157895  0.76870748  0.80530973]
the recall for 10-fold cv is:
[ 0.856  0.88   0.824  0.832  0.608  0.656  0.832  0.672  0.88   0.744]
the avg accuracy is: 0.736522613065
the avg precision is: 0.77995677426
the avg recall is: 0.7784


ii.Why are they different from the results in the previous test?

there are several reasons to cause this result
        1. we do not doing the feature reduction or layer limit for the decision tree, so when we doing the dt training for the all data, we have the overfit which make all the result is 100%.
        2. for the 10-fold part, because we doing the corss validation, so it will decrease the rate of overfit, which we also decrease the accuracy, precasion, recall for the test set.

### 2. Linear Classification

#### a. Use GaussianNB to learn a Naive Bayes classifier to predict highCrime.

i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [146]:
#init the gnb clf
nb_clf = GaussianNB()
nb_clf.fit(clean_X,clean_clf_y)
nb_clf_accuracy = cross_val_score(nb_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
nb_clf_precision = cross_val_score(nb_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
nb_clf_recall = cross_val_score(nb_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(nb_clf_accuracy)
print("the precision for 10 cv is:")
print(nb_clf_precision)
print("the recall for 10 cv is:")
print(nb_clf_recall)
print("the avg accuracy is:",sum(nb_clf_accuracy)/10)
print("the avg precision is:",sum(nb_clf_precision)/10)
print("the avg recall is:",sum(nb_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.775       0.8         0.825       0.79899497  0.70351759  0.65326633
  0.81407035  0.73366834  0.71356784  0.79899497]
the precision for 10 cv is:
[ 0.86363636  0.92929293  0.95        0.92079208  0.94594595  0.86842105
  0.92307692  1.          0.77868852  0.93814433]
the recall for 10 cv is:
[ 0.76   0.736  0.76   0.744  0.56   0.528  0.768  0.576  0.76   0.728]
the avg accuracy is: 0.761608040201
the avg precision is: 0.911799814828
the avg recall is: 0.692


ii.	What are the 10 most predictive features? Why do these make sense (or not)?

In [10]:
nb_main_feature = dict(zip(clean_features,abs(nb_clf.theta_[0] - nb_clf.theta_[1])/(nb_clf.sigma_[0] + nb_clf.sigma_[1])))
for index,w in enumerate(sorted(nb_main_feature, key=nb_main_feature.get, reverse=True)):
    print(w ,nb_main_feature[w])
    if index == 10:
        break

PctKids2Par 5.00131022903
FemalePctDiv 4.75688596283
PctFam2Par 4.59398800551
pctWInvInc 4.39499084626
TotalPctDiv 4.38598607605
PctTeen2Par 3.99935365439
MalePctDivorce 3.95073141349
PctYoungKids2Par 3.64830107618
PctIlleg 3.4698885122
racePctWhite 3.44322831319
PctHousLess3BR 3.29878847562


the result of the main feature is make sense, because according to the form we are using, what we trying to find is the the feature that have the largest class mean difference with low variance. When the result of the (ut - uf)/(sigmat + sigmaf) is larger, it means that they have large mean difference or have smaller variance or both. Which means they have a relatively large gap when using that feature to do the classification. So the main feature is make sense.

iii. How do these results compare with your results from decision trees above?

most main features is similiar with the decision tree, with a slight difference. 

the 10-fold cv result we can find that the accuracy and precision using NB is better than using the decision tree, while the recall of NB is worse than using the DT. The reason is like follows:

what decision tree did is to find the minimum entropy in every step, but it is very easy to make overfit if training all the data at once. And the performance of decision tree is good when the data is very large.
GaussianNB use the maximum likelihood to predict parameters and it assume the features are not related. And it have a good estimate when the data the relatively small.
Back to our training data, because we use the 10-fold cv and the data amount for the decision tree is not large enough, so it have a lower precision and accuracy. And NB have a better result as same time.
And for the recall. NB is not like DT that trying have a minimum entroy, so it make cause more FP or FN when using that classifier, which may make the recall is a little lower than the DT classifier.

#### b.	Use LinearSVC to learn a linear Support Vector Machine model to predict highCrime.

i.	 What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [148]:
#init linear svm
ln_svm_clf = LinearSVC()
ln_svm_clf.fit(clean_X,clean_clf_y)
ln_svm_clf_accuracy = cross_val_score(ln_svm_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
ln_svm_clf_precision = cross_val_score(ln_svm_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
ln_svm_clf_recall = cross_val_score(ln_svm_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(ln_svm_clf_accuracy)
print("the precision for 10 cv is:")
print(ln_svm_clf_precision)
print("the recall for 10 cv is:")
print(ln_svm_clf_recall)
print("the avg accuracy is:",sum(ln_svm_clf_accuracy)/10)
print("the avg precision is:",sum(ln_svm_clf_precision)/10)
print("the avg recall is:",sum(ln_svm_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.79        0.865       0.84        0.80904523  0.69346734  0.69346734
  0.81407035  0.8241206   0.81909548  0.81407035]
the precision for 10 cv is:
[ 0.76774194  0.84507042  0.82517483  0.85950413  0.87209302  0.7962963
  0.84920635  0.95918367  0.80689655  0.87288136]
the recall for 10 cv is:
[ 0.952  0.96   0.944  0.832  0.6    0.688  0.856  0.752  0.936  0.824]
the avg accuracy is: 0.796233668342
the avg precision is: 0.845404856531
the avg recall is: 0.8344


ii.	What are the 10 most predictive features? This can be measured by the absolute feature weights in the model. Why do these make sense (or not)?

In [12]:
ln_svm_feature_value = abs(ln_svm_clf.coef_[0])
ln_svm_main_feature = dict(zip(clean_features,ln_svm_feature_value))
for index,w in enumerate(sorted(ln_svm_main_feature, key=ln_svm_main_feature.get, reverse=True)):
    print(w ,ln_svm_main_feature[w])
    if index == 10:
        break

pctWInvInc 1.88848797592
PersPerOccupHous 1.75512340176
racePctWhite 1.50021823724
PctKids2Par 1.19032762621
RentHighQ 1.06688182158
MalePctDivorce 1.06569629959
NumUnderPov 1.0515475679
NumStreet 1.0191545849
PctOccupMgmtProf 1.01467446351
population 1.0023018298
agePct12t21 0.988272831846


the result is make sense, this is because the final predict for the linear svm is the combination of features with there weights as coefficients, so the bigger the weights, the more influence on the final result. So the features rank by the features weight is make sense.

iii.How do these results compare with your results from decision trees above?

most main features is similiar with the decision tree, with a slight difference. 

the recall, precision and accuracy in linear svm are all better than in the decision tree classifier. this may because the data is not large enought for the decision tree to make a good predict.

### 3. Regression

#### a.	Use LinearRegression to learn a linear model directly predicting the crime rate per capita (ViolentCrimesPerPop).

i.	Using 10-fold cross-validation, what is the estimated mean-squared-error (MSE) of the model?

In [14]:
#init linear regression
LR = LinearRegression()
LR.fit(clean_X,clean_reg_y)
LR_cv_MSE = cross_val_score(LR, clean_X, clean_reg_y, cv=10,scoring='neg_mean_squared_error')
print("the 10-fold CV MSE for linear regression is:")
print(LR_cv_MSE)

the 10-fold CV MSE for linear regression is:
[-0.02983448 -0.02048526 -0.02735939 -0.02320408 -0.01944074 -0.01262621
 -0.01844842 -0.01222962 -0.02183042 -0.01548108]


ii.What is the MSE on the training set (train on all the data then test on it all)?

In [15]:
LR_MSE = mean_squared_error(clean_reg_y, LR.predict(clean_X))
print("the MSE for linear regression is:")
print(LR_MSE)

the MSE for linear regression is:
0.0165167748803


iii. What features are most predictive of a high crime rate? A low crime rate?

In [16]:
LR_main_feature = dict(zip(clean_features,LR.coef_))
#the high crime rate features
for index,w in enumerate(sorted(LR_main_feature, key=LR_main_feature.get, reverse=True)):
    print(w ,LR_main_feature[w])
    if index == 10:
        break

PersPerOccupHous 0.635088116499
PctHousOwnOcc 0.568133209887
MalePctDivorce 0.458517048642
PctRecImmig8 0.432510557653
MedRent 0.372727797711
medFamInc 0.287978874946
PctEmploy 0.248474316085
MalePctNevMarr 0.226727912842
PctPersDenseHous 0.214352564142
OwnOccMedVal 0.212875863654
racepctblack 0.204933833659


In [17]:
#the low crime rate features
for index,w in enumerate(sorted(LR_main_feature, key=LR_main_feature.get, reverse=False)):
    print(w ,LR_main_feature[w])
    if index == 10:
        break

PctPersOwnOccup -0.675694478803
TotalPctDiv -0.561924314415
whitePerCap -0.351015774441
PctKids2Par -0.322651276496
OwnOccLowQuart -0.308170219193
numbUrban -0.296442543178
PersPerRentOccHous -0.254571662716
RentLowQ -0.234751516303
agePct12t29 -0.229217836999
PctRecImmig5 -0.218221007355
pctWWage -0.201573959743


PersPerOccupHous predict a high crime rate, PctPersOwnOccup predict a low crime rate

#### b.	Now use Ridge regression to reduce the amount of overfitting, using RidgeCV to pick the best alpha from among (10, 1, 0.1, 0.01, and 0.001).

i.	What is the estimated MSE of the model under 10-fold CV?

In [18]:
#we use the default parameter(alpha = 1.0) to init first
RG = Ridge()
RG.fit(clean_X, clean_reg_y)
RG_cv_MSE = cross_val_score(RG, clean_X, clean_reg_y, cv=10,scoring='neg_mean_squared_error')
print("the 10-fold cv MSE for ridge regression is:")
print(RG_cv_MSE)

the 10-fold cv MSE for ridge regression is:
[-0.0295138  -0.01972075 -0.02795035 -0.02311508 -0.01872138 -0.0120445
 -0.01780565 -0.01212225 -0.02191791 -0.01503854]


ii.What is the MSE on the training set (train on all the data then test on it all)?

In [19]:
RG_MSE = mean_squared_error(clean_reg_y, RG.predict(clean_X))
print("the MSE for ridge regression is:")
print(RG_MSE)

the MSE for ridge regression is:
0.0167635291552


iii.What is the best alpha?

In [20]:
RG_cv = RidgeCV(alphas = np.array([10,1,0.1,0.01,0.001]), cv = 10)
RG_cv.fit(clean_X,clean_reg_y)
print("the best alpha is:")
print(RG_cv.alpha_)

the best alpha is:
1.0


because the default alpha is also 1.0, so we don't need to calculate the 10-fold CV MSE and the MSE

iv.What does this say about the amount of overfitting in linear regression for this problem?

we can find that the 10-fold MSE in ridge regression is less than using the linear regression. This is because the ridge regression have give more penality to the unimportant features, which make the overfit of the ridge regression decrease. So the ridge regression have a smaller 10-fold cv MSE than the linear regression's 10-fold cv MSE.

#### c.	Now use polynomial features to do quadratic (second-order) polynomial regression.

we need to prepare the data again to do the polynomial regression.

In [21]:
#using the PolynomialFeatures to transform the data
poly = PolynomialFeatures(degree=2)
poly_clean_X = poly.fit_transform(clean_X)

i.What is the estimated MSE of the model under 10-fold CV?

In [23]:
#init the poly regression
poly_LR = LinearRegression()
poly_LR.fit(poly_clean_X,clean_reg_y)
poly_LR_cv_MSE = cross_val_score(poly_LR, poly_clean_X, clean_reg_y, cv=10,scoring='neg_mean_squared_error')
print("the 10-fold cv poly regression MSE is:")
print(poly_LR_cv_MSE)

the 10-fold cv poly regression MSE is:
[-0.15479254 -0.1498775  -0.19852639 -0.1021161  -0.07857951 -0.1410196
 -0.09207064 -0.0737748  -0.1692134  -0.13903143]


ii.What is the MSE on the training set (train on all the data then test on it all)?

In [24]:
poly_LR_MSE = mean_squared_error(clean_reg_y, poly_LR.predict(poly_clean_X))
print("the poly regression MSE is:")
print(poly_LR_MSE)

the poly regression MSE is:
6.91579443622e-29


iii.Does this mean the quadratic model is better than the linear model for this problem?

the result is not better than the linear regression model, because it is overfit, it makes the 10-fold cv MSE larger than the linear regression one, and because of the overfit, it make the full data MSE very very small.

### 4. Dirty Data

Repeat the decision tree learning question for the full (non-clean) data set and present the results.

#### a. get the crime rate and calculate the percentage of positive and negative example

In [82]:
#the highCrime is the same as clean_clf_y
full_highCrime = list(full_clf_y)
#the positive number and neg number is 
full_pos = full_highCrime.count(1)
full_neg = full_highCrime.count(0)
#the percentage
print("the positve percentage is:")
print(full_pos/len(full_highCrime))
print("the negative percentage is:")
print(full_neg/len(full_highCrime))

the positve percentage is:
0.627382146439318
the negative percentage is:
0.37261785356068206


#### b.  Use DecisionTreeClassifier to learn a decision tree to predict highCrime

In [96]:
#init the full DT clf
full_dt_clf = DecisionTreeClassifier()
#fit the data
full_dt_clf.fit(full_X,full_clf_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

i. the accuracy, precision and recall are as follows:

In [97]:
full_answer = full_dt_clf.predict_proba(full_X)[:,1]  
full_accuracy = accuracy_score(full_clf_y, full_dt_clf.predict(full_X))  
print("the accuracy is ",full_accuracy)
print(classification_report(full_clf_y,full_answer, target_names = ['negative', 'positive']))

the accuracy is  1.0
             precision    recall  f1-score   support

   negative       1.00      1.00      1.00       743
   positive       1.00      1.00      1.00      1251

avg / total       1.00      1.00      1.00      1994



ii. now let's find the main features according to the gini importance

In [98]:
full_main_feature = dict(zip(full_features,full_dt_clf.feature_importances_))
for index,w in enumerate(sorted(full_main_feature, key=full_main_feature.get, reverse=True)):
    print(w ,full_main_feature[w])
    if index == 10:
        break

PctKids2Par 0.358795967643
racePctWhite 0.086480363916
racePctHisp 0.0505171773405
MedRent 0.0191605094703
PctEmplManu 0.0191288131366
PctLess9thGrade 0.0187335227013
PctEmploy 0.0162205304653
PctSpeakEnglOnly 0.0149754941779
blackPerCap 0.0148174881971
PctImmigRec10 0.0143954680319
PctSameState85 0.012215528406


The above result is make sense, because the decision tree is create by get the minimum entropy at every decision, and from the data we can find that if we order the data with the main features above, they are all high related to the crime rate. So the above result make sense.

#### c. Now apply cross-validation (cross_val_score) to do 10-fold cross-validation to estimate the out-of-training accuracy of decision tree learning for this task.

i. What are the 10-fold cross-validation accuracy, precision, and recall?

In [99]:
full_ten_fold_accuracy = cross_val_score(full_dt_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
full_ten_fold_precision = cross_val_score(full_dt_clf, full_X, full_clf_y, cv=10,scoring='precision')
full_ten_fold_recall = cross_val_score(full_dt_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(full_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(full_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(full_ten_fold_recall)

the accuracy for 10-fold cv is:
[ 0.80597015  0.745       0.735       0.75376884  0.72361809  0.75376884
  0.80904523  0.75879397  0.74371859  0.74371859]
the precision for 10-fold cv is:
[ 0.85        0.81746032  0.8         0.80645161  0.79831933  0.81512605
  0.8699187   0.80672269  0.79130435  0.7804878 ]
the recall for 10-fold cv is:
[ 0.81746032  0.808       0.744       0.848       0.768       0.784       0.84
  0.752       0.76        0.784     ]


ii.Why are they different from the results in the previous test?

there are several reasons to cause this result
        1. we do not doing the feature reduction or layer limit for the decision tree, so when we doing the dt training for the all data, we have the overfit which make all the result is 100%.
        2. for the 10-fold part, because we doing the corss validation, so it will decrease the rate of overfit, which we also decrease the accuracy, precasion, recall for the test set.

#### additional question :
#### a.	Are the CV results better or worse? What does this say about the effect of missing values?

the CV result is better than the clean data one, this means that some parameters which misssing value are also important features that influence the predict of the decision tree classifier

### 5.	Teams 

because we are a two-member team, so we will do the part a, and part b will for extra credits

####  a. If you are working in a team of two people:

i.	Experiment with two learning methods other than those described above (one can be a non-linear kernel for SVM) for the classification problem, explaining clearly what you did. Show CV results for both the clean and full datasets.

###### using the SGDClassifier first

In [100]:
#init the sgd_clf for clean data
clean_sgd_clf = SGDClassifier()
sgd_clean_ten_fold_accuracy = cross_val_score(clean_sgd_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
sgd_clean_ten_fold_precision = cross_val_score(clean_sgd_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
sgd_clean_ten_fold_recall = cross_val_score(clean_sgd_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(sgd_clean_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(sgd_clean_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(sgd_clean_ten_fold_recall)

the accuracy for 10-fold cv is:
[ 0.74        0.845       0.84        0.85427136  0.65326633  0.70854271
  0.81909548  0.87437186  0.82914573  0.72361809]
the precision for 10-fold cv is:
[ 0.70833333  0.7251462   0.7852349   0.69273743  0.96226415  0.75172414
  0.9245283   1.          0.89247312  0.76      ]
the recall for 10-fold cv is:
[ 0.88   0.592  1.     0.992  0.424  0.648  0.824  0.736  0.792  0.648]


In [101]:
#init the sgd_clf for full data
full_sgd_clf = SGDClassifier()
sgd_full_ten_fold_accuracy = cross_val_score(full_sgd_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
sgd_full_ten_fold_precision = cross_val_score(full_sgd_clf, full_X, full_clf_y, cv=10,scoring='precision')
sgd_full_ten_fold_recall = cross_val_score(full_sgd_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(sgd_full_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(sgd_full_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(sgd_full_ten_fold_recall)

the accuracy for 10-fold cv is:
[ 0.64179104  0.805       0.795       0.72864322  0.79899497  0.79396985
  0.86934673  0.84422111  0.79899497  0.71356784]
the precision for 10-fold cv is:
[ 0.67021277  0.73053892  0.65608466  0.80985915  0.94186047  0.72093023
  0.82191781  0.8699187   0.98684211  0.88596491]
the recall for 10-fold cv is:
[ 1.     0.672  0.76   0.744  0.912  0.736  0.592  0.888  0.632  0.912]


###### using the RandomForestClassifier

In [102]:
#init the rf_clf for clean data
clean_rf_clf = RandomForestClassifier()
rf_clean_ten_fold_accuracy = cross_val_score(clean_rf_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
rf_clean_ten_fold_precision = cross_val_score(clean_rf_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
rf_clean_ten_fold_recall = cross_val_score(clean_rf_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(rf_clean_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(rf_clean_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(rf_clean_ten_fold_recall)

the accuracy for 10-fold cv is:
[ 0.81        0.89        0.81        0.79899497  0.71356784  0.73869347
  0.82914573  0.77889447  0.82914573  0.81407035]
the precision for 10-fold cv is:
[ 0.75        0.86507937  0.84558824  0.864       0.86170213  0.87368421
  0.82706767  0.9         0.85294118  0.86065574]
the recall for 10-fold cv is:
[ 0.912  0.928  0.896  0.864  0.616  0.656  0.856  0.672  0.912  0.808]


In [103]:
#init the rf_clf for full data
full_rf_clf = RandomForestClassifier()
rf_full_ten_fold_accuracy = cross_val_score(full_rf_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
rf_full_ten_fold_precision = cross_val_score(full_rf_clf, full_X, full_clf_y, cv=10,scoring='precision')
rf_full_ten_fold_recall = cross_val_score(full_rf_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(rf_full_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(rf_full_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(rf_full_ten_fold_recall)

the accuracy for 10-fold cv is:
[ 0.83084577  0.845       0.795       0.7638191   0.79396985  0.78894472
  0.81909548  0.82914573  0.81407035  0.84924623]
the precision for 10-fold cv is:
[ 0.875       0.828125    0.83969466  0.84552846  0.86956522  0.85217391
  0.8907563   0.89473684  0.85950413  0.85950413]
the recall for 10-fold cv is:
[ 0.84920635  0.848       0.848       0.792       0.824       0.784       0.792
  0.84        0.832       0.848     ]


ii.	What method gives the best results?

In [104]:
print("the avg accuracy,precision and recall for sgd_clf with clean data are:")
print(sum(sgd_clean_ten_fold_accuracy)/10,sum(sgd_clean_ten_fold_precision)/10,sum(sgd_clean_ten_fold_recall)/10)
print("the avg accuracy,precision and recall for sgd_clf with full data are:")
print(sum(sgd_full_ten_fold_accuracy)/10,sum(sgd_full_ten_fold_precision)/10,sum(sgd_full_ten_fold_recall)/10)
print("the avg accuracy,precision and recall for rf_clf with clean data are:")
print(sum(rf_clean_ten_fold_accuracy)/10,sum(rf_clean_ten_fold_precision)/10,sum(rf_clean_ten_fold_recall)/10)
print("the avg accuracy,precision and recall for rf_clf with full data are:")
print(sum(rf_full_ten_fold_accuracy)/10,sum(rf_full_ten_fold_precision)/10,sum(rf_full_ten_fold_recall)/10)

the avg accuracy,precision and recall for sgd_clf with clean data are:
0.788731155779 0.82024415707 0.7536
the avg accuracy,precision and recall for sgd_clf with full data are:
0.778952973824 0.809412972175 0.7848
the avg accuracy,precision and recall for rf_clf with clean data are:
0.801251256281 0.850071852191 0.812
the avg accuracy,precision and recall for rf_clf with full data are:
0.812913722843 0.86145886513 0.825720634921


from the result above we can find that when we using the full data, we can get a better result in SGD classifier and in random forest classifier

Besides, we can find that the random forest classifier have a better result than the SGD classifier.

iii.What feature(s) seem to be most consistently predictive of high crime rates? How reliable is this conclusion?

we can use SGD coef to get the features can predict high crime rate

In [105]:
full_sgd_clf.fit(full_X,full_clf_y)
full_sgd_main_feature = dict(zip(full_features,full_sgd_clf.coef_[0]))
for index,w in enumerate(sorted(full_sgd_main_feature, key=full_sgd_main_feature.get, reverse=True)):
    print(w ,full_sgd_main_feature[w])
    if index == 10:
        break

racepctblack 18.1238034461
racePctHisp 10.5296745373
MalePctDivorce 10.2288266934
PctIlleg 9.16218433768
TotalPctDiv 8.24140760325
PctSameCity85 8.03172577263
PctPersDenseHous 7.82204394202
OtherPerCap 7.57589570608
FemalePctDiv 5.9075576625
HousVacant 5.55201021059
MedRentPctHousInc 5.43349439329


we can also use random forest feature importance to get the features have heavy effect on the crime rate, but we cannot get weather the feature have positive influence or negative influence

In [110]:
full_rf_clf.fit(full_X,full_clf_y)
full_rf_main_feature = dict(zip(full_features,full_rf_clf.feature_importances_))
print("the main features of random forest for the first test")
for index,w in enumerate(sorted(full_rf_main_feature, key=full_rf_main_feature.get, reverse=True)):
    print(w ,full_rf_main_feature[w])
    if index == 10:
        break

the main features of random forest for the first test
FemalePctDiv 0.0820288699727
PctFam2Par 0.0701253187026
PctYoungKids2Par 0.0645716820006
PctPopUnderPov 0.0564569707245
NumIlleg 0.046618267516
PctIlleg 0.0271528877738
PctPersDenseHous 0.0262136831012
racepctblack 0.0254071802517
racePctWhite 0.0241139563332
racePctHisp 0.0184378345414
PctLargHouseFam 0.0183883871743


In [111]:
full_rf_clf.fit(full_X,full_clf_y)
full_rf_main_feature = dict(zip(full_features,full_rf_clf.feature_importances_))
print("the main features of random forest for the second test")
for index,w in enumerate(sorted(full_rf_main_feature, key=full_rf_main_feature.get, reverse=True)):
    print(w ,full_rf_main_feature[w])
    if index == 10:
        break

the main features of random forest for the second test
PctKids2Par 0.0950173078848
racePctWhite 0.0790165718385
FemalePctDiv 0.0785059153857
pctWInvInc 0.0320892894463
PctPersOwnOccup 0.0298088513386
NumUnderPov 0.0271243409871
PctPersDenseHous 0.0234306880347
PctHousOwnOcc 0.0204980193541
PctTeen2Par 0.019813402429
PctIlleg 0.0193924714482
PctFam2Par 0.0175810364368


actually we can find that the main features we get from the random forest is not stable.
Although the random forest can have a good predict of the crime, the result of main features in random forest is not good.

the result of the main features using SGD_classifier seems stable. Because the SGD_classifier using the Gradient Descent to find the best weight for every parameter. Hence the result should be reliable.

But, it also have limits, because our data is not enough(only nearly 2K), we can only confirm that the random forest and SGD can fit our training data well.

back to our data, the most important feature that predict high crime is racepctblack

### 6. Extra Credit

#### a. Do a team requirement (above) that your team is not already required to do.

i. Do the requirement for two person teams.

the result is already show above

ii. Devise a method to find the most useful threshold for dividing high crime areas from low crime areas (i.e., discretizing XXX to compute highCrime). Define clearly what you mean by “useful”.

we can using k-means algorithm to find the threshold

In [134]:
km = KMeans(n_clusters=2)
tmp = full_reg_y.copy()
tmp = tmp.values.reshape(-1,1)
km.fit(tmp)
threshold = sum(km.cluster_centers_)/2
print("the new threshold is:", threshold)

the new threshold is: [ 0.38541353]


this threshold is useful because it using the k-means algorithm to find the cluster centers and using the middle point as the threshold. If the algorithm is correct, it should be able to increase the accuracy.

iii. Show CV results for both the clean and full datasets for at least three different classification methods above.

##### before doing the classification, we need to create the new label for the new classification.

In [140]:
new_clean_clf_y = clean_reg_y.copy()
new_full_clf_y = full_reg_y.copy()
for index, item in enumerate(new_clean_clf_y):
    if (item > threshold):
        new_clean_clf_y[index] = 1
    else:
        new_clean_clf_y[index] = 0
for index, item in enumerate(new_full_clf_y):
    if (item > threshold):
        new_full_clf_y[index] = 1
    else:
        new_full_clf_y[index] = 0

###### we will using decision tree, GaussianNB and LinearSVC to test the result

###### for the clean data:

In [143]:
#using the dt
new_dt_clf = DecisionTreeClassifier()
new_ten_fold_accuracy = cross_val_score(new_dt_clf, clean_X, new_clean_clf_y, cv=10,scoring='accuracy')
new_ten_fold_precision = cross_val_score(new_dt_clf, clean_X, new_clean_clf_y, cv=10,scoring='precision')
new_ten_fold_recall = cross_val_score(new_dt_clf, clean_X, new_clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(new_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(new_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(new_ten_fold_recall)
print("the avg accuracy is:",sum(new_ten_fold_accuracy)/10)
print("the avg precision is:",sum(new_ten_fold_precision)/10)
print("the avg recall is:",sum(new_ten_fold_recall)/10)

the accuracy for 10-fold cv is:
[ 0.75124378  0.84        0.70351759  0.83919598  0.83417085  0.7839196
  0.85929648  0.85427136  0.61809045  0.73366834]
the precision for 10-fold cv is:
[ 0.44444444  0.61111111  0.38888889  0.6969697   0.55        0.5
  0.63414634  0.65789474  0.3164557   0.425     ]
the recall for 10-fold cv is:
[ 0.58536585  0.55        0.575       0.55        0.6         0.525       0.675
  0.625       0.625       0.375     ]
the avg accuracy is: 0.781737443436
the avg precision is: 0.522491091592
the avg recall is: 0.568536585366


In [144]:
#using the gaussian NB
new_nb_clf = GaussianNB()
new_nb_clf_accuracy = cross_val_score(new_nb_clf, clean_X, new_clean_clf_y, cv=10,scoring='accuracy')
new_nb_clf_precision = cross_val_score(new_nb_clf, clean_X, new_clean_clf_y, cv=10,scoring='precision')
new_nb_clf_recall = cross_val_score(new_nb_clf, clean_X, new_clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(new_nb_clf_accuracy)
print("the precision for 10 cv is:")
print(new_nb_clf_precision)
print("the recall for 10 cv is:")
print(new_nb_clf_recall)
print("the avg accuracy is:",sum(new_nb_clf_accuracy)/10)
print("the avg precision is:",sum(new_nb_clf_precision)/10)
print("the avg recall is:",sum(new_nb_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.75124378  0.825       0.84422111  0.88442211  0.85427136  0.83919598
  0.87437186  0.87939698  0.72361809  0.83417085]
the precision for 10 cv is:
[ 0.44        0.54237288  0.57142857  0.71794872  0.62222222  0.6
  0.63636364  0.69047619  0.41176471  0.57142857]
the recall for 10 cv is:
[ 0.80487805  0.8         0.9         0.7         0.7         0.6         0.875
  0.725       0.875       0.7       ]
the avg accuracy is: 0.83099121228
the avg precision is: 0.580400549711
the avg recall is: 0.767987804878


In [147]:
#using linear svm
new_ln_svm_clf = LinearSVC()
new_ln_svm_clf_accuracy = cross_val_score(new_ln_svm_clf, clean_X, new_clean_clf_y, cv=10,scoring='accuracy')
new_ln_svm_clf_precision = cross_val_score(new_ln_svm_clf, clean_X, new_clean_clf_y, cv=10,scoring='precision')
new_ln_svm_clf_recall = cross_val_score(new_ln_svm_clf, clean_X, new_clean_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(new_ln_svm_clf_accuracy)
print("the precision for 10 cv is:")
print(new_ln_svm_clf_precision)
print("the recall for 10 cv is:")
print(new_ln_svm_clf_recall)
print("the avg accuracy is:",sum(new_ln_svm_clf_accuracy)/10)
print("the avg precision is:",sum(new_ln_svm_clf_precision)/10)
print("the avg recall is:",sum(new_ln_svm_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.85074627  0.89        0.8040201   0.86432161  0.85929648  0.85929648
  0.89949749  0.89447236  0.73869347  0.81909548]
the precision for 10 cv is:
[ 0.63414634  0.76470588  0.51282051  0.72413793  0.66666667  0.8
  0.83333333  0.88        0.41176471  0.6       ]
the recall for 10 cv is:
[ 0.63414634  0.65        0.5         0.525       0.6         0.4         0.625
  0.55        0.7         0.3       ]
the avg accuracy is: 0.847943973599
the avg precision is: 0.682757537355
the avg recall is: 0.548414634146


###### for the dirty data:

In [150]:
#using the dt
full_new_dt_clf = DecisionTreeClassifier()
full_new_ten_fold_accuracy = cross_val_score(full_new_dt_clf, full_X, new_full_clf_y, cv=10,scoring='accuracy')
full_new_ten_fold_precision = cross_val_score(full_new_dt_clf, full_X, new_full_clf_y, cv=10,scoring='precision')
full_new_ten_fold_recall = cross_val_score(full_new_dt_clf, full_X, new_full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(full_new_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(full_new_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(full_new_ten_fold_recall)
print("the avg accuracy is:",sum(full_new_ten_fold_accuracy)/10)
print("the avg precision is:",sum(full_new_ten_fold_precision)/10)
print("the avg recall is:",sum(full_new_ten_fold_recall)/10)

the accuracy for 10-fold cv is:
[ 0.82089552  0.78        0.835       0.8040201   0.8241206   0.81909548
  0.83919598  0.81407035  0.87437186  0.84422111]
the precision for 10-fold cv is:
[ 0.51351351  0.40909091  0.53846154  0.58333333  0.57894737  0.48888889
  0.56521739  0.56097561  0.65        0.62162162]
the recall for 10-fold cv is:
[ 0.46341463  0.475       0.75        0.7         0.525       0.55        0.725
  0.55        0.7         0.575     ]
the avg accuracy is: 0.825499099977
the avg precision is: 0.551005017439
the avg recall is: 0.601341463415


In [152]:
#using the gaussian NB
full_new_nb_clf = GaussianNB()
full_new_nb_clf_accuracy = cross_val_score(full_new_nb_clf, full_X, new_full_clf_y, cv=10,scoring='accuracy')
full_new_nb_clf_precision = cross_val_score(full_new_nb_clf, full_X, new_full_clf_y, cv=10,scoring='precision')
full_new_nb_clf_recall = cross_val_score(full_new_nb_clf, full_X, new_full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(full_new_nb_clf_accuracy)
print("the precision for 10 cv is:")
print(full_new_nb_clf_precision)
print("the recall for 10 cv is:")
print(full_new_nb_clf_recall)
print("the avg accuracy is:",sum(full_new_nb_clf_accuracy)/10)
print("the avg precision is:",sum(full_new_nb_clf_precision)/10)
print("the avg recall is:",sum(full_new_nb_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.7960199   0.815       0.805       0.83417085  0.84924623  0.85427136
  0.84422111  0.7839196   0.81407035  0.85929648]
the precision for 10 cv is:
[ 0.5         0.53658537  0.51111111  0.56603774  0.63157895  0.62222222
  0.61538462  0.46666667  0.53333333  0.65789474]
the recall for 10 cv is:
[ 0.58536585  0.55        0.575       0.75        0.6         0.7         0.6
  0.525       0.6         0.625     ]
the avg accuracy is: 0.82552158804
the avg precision is: 0.564081473463
the avg recall is: 0.611036585366


In [153]:
#using linear svm
full_new_ln_svm_clf = LinearSVC()
full_new_ln_svm_clf_accuracy = cross_val_score(full_new_ln_svm_clf, full_X, new_full_clf_y, cv=10,scoring='accuracy')
full_new_ln_svm_clf_precision = cross_val_score(full_new_ln_svm_clf, full_X, new_full_clf_y, cv=10,scoring='precision')
full_new_ln_svm_clf_recall = cross_val_score(full_new_ln_svm_clf, full_X, new_full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(full_new_ln_svm_clf_accuracy)
print("the precision for 10 cv is:")
print(full_new_ln_svm_clf_precision)
print("the recall for 10 cv is:")
print(full_new_ln_svm_clf_recall)
print("the avg accuracy is:",sum(full_new_ln_svm_clf_accuracy)/10)
print("the avg precision is:",sum(full_new_ln_svm_clf_precision)/10)
print("the avg recall is:",sum(full_new_ln_svm_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.85572139  0.885       0.88        0.88944724  0.86432161  0.90954774
  0.90452261  0.86934673  0.87939698  0.87939698]
the precision for 10 cv is:
[ 0.77272727  0.74285714  0.76666667  0.76470588  0.6969697   0.86666667
  0.81818182  0.75        0.72222222  0.76666667]
the recall for 10 cv is:
[ 0.41463415  0.65        0.575       0.65        0.575       0.65        0.675
  0.525       0.65        0.575     ]
the avg accuracy is: 0.881670129253
the avg precision is: 0.766766403531
the avg recall is: 0.593963414634


iv. How are these results similar and different from the previous results (with a fixed threshold of 0.1). What does this say about how to approach such a problem.

we can find that the result is different from the previous results. Using the new threshold increase the accuracy while decrease the precision and recall. This is because the new threshold have increase the number of negative example. Which may cause the decrease of precision and recall.

overall, the new threshold is relatively good, because it's increase the accuracy, which is the most important parameter in most cases.

####  b. Experiment with other learning methods such as polynomial or other kernels in SVM, decision forests, boosting, etc. and show your results. Make sure to explain clearly what you did.

###### we analysis the classification method first

using the method to deal the clean data first

1.we can using the adabooster method to deal with the data

In [154]:
#init the adb_clf with clean data
clean_adb_clf = AdaBoostClassifier()
adb_clean_ten_fold_accuracy = cross_val_score(clean_adb_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
adb_clean_ten_fold_precision = cross_val_score(clean_adb_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
adb_clean_ten_fold_recall = cross_val_score(clean_adb_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the avg accuracy is:",sum(adb_clean_ten_fold_accuracy)/10)
print("the avg precision is:",sum(adb_clean_ten_fold_precision)/10)
print("the avg recall is:",sum(adb_clean_ten_fold_recall)/10)

the avg accuracy is: 0.782668341709
the avg precision is: 0.837085111098
the avg recall is: 0.8216


2.we can also using the GradientBoostingClassifier to deal with data

In [156]:
#init the gb_clf with clean data
clean_gb_clf = GradientBoostingClassifier()
gb_clean_ten_fold_accuracy = cross_val_score(clean_gb_clf, clean_X, clean_clf_y, cv=10,scoring='accuracy')
gb_clean_ten_fold_precision = cross_val_score(clean_gb_clf, clean_X, clean_clf_y, cv=10,scoring='precision')
gb_clean_ten_fold_recall = cross_val_score(clean_gb_clf, clean_X, clean_clf_y, cv=10,scoring='recall')
print("the avg accuracy is:",sum(gb_clean_ten_fold_accuracy)/10)
print("the avg precision is:",sum(gb_clean_ten_fold_precision)/10)
print("the avg recall is:",sum(gb_clean_ten_fold_recall)/10)

the avg accuracy is: 0.798213567839
the avg precision is: 0.850568409405
the avg recall is: 0.8336


3.we have implement the random forest at above section, the result is like below:

In [157]:
print("the avg accuracy is:",sum(rf_clean_ten_fold_accuracy)/10)
print("the avg precision is:",sum(rf_clean_ten_fold_precision)/10)
print("the avg recall is:",sum(rf_clean_ten_fold_recall)/10)

the avg accuracy is: 0.801251256281
the avg precision is: 0.850071852191
the avg recall is: 0.812


4.we have also implement the SGD_clf at above, the result is like below:

In [158]:
print("the avg accuracy is:",sum(sgd_clean_ten_fold_accuracy)/10)
print("the avg precision is:",sum(sgd_clean_ten_fold_precision)/10)
print("the avg recall is:",sum(sgd_clean_ten_fold_recall)/10)

the avg accuracy is: 0.788731155779
the avg precision is: 0.82024415707
the avg recall is: 0.7536


we can deal the full data using the above method too:

In [159]:
#init the full adb_clf
full_adb_clf = AdaBoostClassifier()
adb_full_ten_fold_accuracy = cross_val_score(full_adb_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
adb_full_ten_fold_precision = cross_val_score(full_adb_clf, full_X, full_clf_y, cv=10,scoring='precision')
adb_full_ten_fold_recall = cross_val_score(full_adb_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the avg accuracy is:",sum(adb_full_ten_fold_accuracy)/10)
print("the avg precision is:",sum(adb_full_ten_fold_precision)/10)
print("the avg recall is:",sum(adb_full_ten_fold_recall)/10)

the avg accuracy is: 0.820438898472
the avg precision is: 0.85466453235
the avg recall is: 0.861688888889


In [160]:
#init the full gb_clf
full_gb_clf = GradientBoostingClassifier()
gb_full_ten_fold_accuracy = cross_val_score(full_gb_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
gb_full_ten_fold_precision = cross_val_score(full_gb_clf, full_X, full_clf_y, cv=10,scoring='precision')
gb_full_ten_fold_recall = cross_val_score(full_gb_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the avg accuracy is:",sum(gb_full_ten_fold_accuracy)/10)
print("the avg precision is:",sum(gb_full_ten_fold_precision)/10)
print("the avg recall is:",sum(gb_full_ten_fold_recall)/10)

the avg accuracy is: 0.837989224731
the avg precision is: 0.873403006091
the avg recall is: 0.868095238095


In [161]:
#using the above full rf_clf
print("the avg accuracy is:",sum(rf_full_ten_fold_accuracy)/10)
print("the avg precision is:",sum(rf_full_ten_fold_precision)/10)
print("the avg recall is:",sum(rf_full_ten_fold_recall)/10)

the avg accuracy is: 0.812913722843
the avg precision is: 0.86145886513
the avg recall is: 0.825720634921


In [162]:
#using the above full sgd_clf
print("the avg accuracy is:",sum(sgd_full_ten_fold_accuracy)/10)
print("the avg precision is:",sum(sgd_full_ten_fold_precision)/10)
print("the avg recall is:",sum(sgd_full_ten_fold_recall)/10)

the avg accuracy is: 0.778952973824
the avg precision is: 0.809412972175
the avg recall is: 0.7848


i.What method gives the best results?

at first we can find that we can get a better result when using the full data.

we can find that when we using the GradientBoostingClassifier, we can get a really good result, it have high precision, accuracy and recall.

ii. What feature(s) seem to be most consistently predictive of high crime rates? How reliable is this conclusion?

In [163]:
full_gb_clf.fit(full_X,full_clf_y)
full_gb_main_feature = dict(zip(full_features,full_gb_clf.feature_importances_))
print("the main features of random forest for the second test")
for index,w in enumerate(sorted(full_gb_main_feature, key=full_gb_main_feature.get, reverse=True)):
    print(w ,full_gb_main_feature[w])
    if index == 10:
        break

the main features of random forest for the second test
PctKids2Par 0.0791857847178
racePctWhite 0.0520797883371
FemalePctDiv 0.0308962577772
PctImmigRec10 0.0248180889751
PctBornSameState 0.023418099002
NumIlleg 0.0221792618535
PctImmigRec5 0.0208748669095
pctWInvInc 0.0208508704835
PctVacMore6Mos 0.020319104558
TotalPctDiv 0.0195249459073
HousVacant 0.0193909432843


from the result above, we know that the PctKids2Par have a huge influence of crime rate, but we don's know if it have positive or negative influence to the high crime rate. While from the analysis in the regresssion part, we can find that this feature have a negative influence to the high crime rate. 

the feature is fit well for the training data we used, and according to the result of 10-fold CV, it should be fine when deal with other data. But we need to notice that our training data is only nearly 2k, so if we can gathering more data, we will get a better result.

###### we analysis the regression method next

In [164]:
# we using the ard regression to train the data and get the mse
full_ard_reg = ARDRegression()
full_ard_reg.fit(full_X,full_reg_y)
full_ard_reg_MSE = mean_squared_error(full_reg_y, full_ard_reg.predict(full_X))
print("the ard regression MSE is:")
print(full_ard_reg_MSE)

the ard regression MSE is:
0.016838160795


In [165]:
# we can also using the Least Angle Regression
full_lars_reg = Lars()
full_lars_reg.fit(full_X,full_reg_y)
full_lars_reg_MSE = mean_squared_error(full_reg_y, full_lars_reg.predict(full_X))
print("the lars regression MSE is:")
print(full_lars_reg_MSE)

the lars regression MSE is:
4.0504671199


i. What method gives the best results?

for the regression part, we can find that the MSE of ardregression is similiar with linear regression, while the lars regression donnot have a good MSE.
combine with the regressison model at the secone part, we find that the ard regression give a better result.

ii.What feature(s) seem to be most consistently predictive of high crime rates? How reliable is this conclusion?

In [179]:
full_ard_reg_main_feature = dict(zip(full_features,full_ard_reg.coef_))
print("the main features of random forest for the second test")
for index,w in enumerate(sorted(full_ard_reg_main_feature, key=full_ard_reg_main_feature.get, reverse=True)):
    print(w ,full_ard_reg_main_feature[w])
    if index == 10:
        break

the main features of random forest for the second test
population 2.55485909669
RentHighQ 0.619174759043
PctOccupMgmtProf 0.389099531506
PersPerRentOccHous 0.304379272636
racePctHisp 0.287399607682
PctTeen2Par 0.267707033267
MalePctDivorce 0.251091850066
RacialMatchCommPol 0.242476105556
perCapInc 0.158897857262
OtherPerCap 0.136565399234
pctUrban 0.127986059805


the result is make some sense, the population is a feature of crime, but because it using the NB, actually it's assumr all the features are irrelevant. So it may cause some mistake. Besides, our training set is only 2k smaple, so we can only confirm that this model can do pretty good in our trainning set. If we can training more data, the result should be better.

####  c. Find other data sets to combine with this data set that might improve results. This may include weather data (from NOAA), other demographics data from the US Census, etc. One good source for data sets is http://data.gov

i. Explain precisely what you did to combine the datasets.

before i trying to find the data, i have search the original data form, i found that it contains the state number
, community name. But when i trying to find the state naming using the community name, i find for a same community name, contains a lot of state. So i am trying to figure out what state it is in the data. By searching the documents 
of the data, i found the state number is assign by the State FIPS Code

the corresponding order are like below:
1 Alabama
2 Alaska
3 American Samoa
4 Arizona
5 Arkansas
6 California
8 Colorado
9 Connecticut
10 Delaware
11 District of Columbia
12 Florida
13 Georgia
14 Guam
15 Hawaii
16 Idaho
17 Illinois
18 Indiana
19 Iowa
20 Kansas
21 Kentucky
22 Louisiana
23 Maine
24 Maryland
25 Massachusetts
26 Michigan
27 Minnesota
28 Mississippi
29 Missouri
30 Montana
31 Nebraska
32 Nevada
33 New Hampshire
34 New Jersey
35 New Mexico
36 New York
37 North Carolina
38 North Dakota
39 Ohio
40 Oklahoma
41 Oregon
42 Pennsylvania
43 Puerto Rico
44 Rhode Island
45 South Carolina
46 South Dakota
47 Tennessee
48 Texas
49 Utah
50 Vermont
51 Virginia
52 Virgin Islands
53 Washington
54 West Virginia
55 Wisconsin
56 Wyoming

then, i am trying to find the data from the data.gov. By taking some effort, i have find the data set called "Community Health Status Indicators (CHSI) to Combat Obesity, Heart Disease and Cancer", the link is below:
https://catalog.data.gov/dataset/community-health-status-indicators-chsi-to-combat-obesity-heart-disease-and-cancer

i am using the csv file called "SUMMARYMEASURESOFHEALTH.csv" to combine my original data.
because the cimmunity name in our original data contains some suffixes like "city,town,township,borough,village".
so i deal with this suffix first.

In [329]:
#read the clean data using pandas
dem = pd.read_csv("SUMMARYMEASURESOFHEALTH.csv")

tmp_full_X = pd.read_csv("communities-crime-full.csv")
tmp_full_X['communityname'] = tmp_full_X['communityname'].str.replace(r'(city|borough|City|township|village|town)$','')
tmp_full_X['communityname'] = tmp_full_X['communityname'].str.replace(r'City$','')
new_full_X = tmp_full_X.merge(dem, left_on = ['state','communityname'],right_on = ['State_FIPS_Code','CHSI_County_Name'],how='inner',suffixes=('_original', '_addition'))
print("if we using the inner join, we can find the smaple number become:", len(new_full_X))


if we using the inner join, we can find the smaple number become: 162


so, we using the left outer join, hope the few new attr can make some help

In [330]:
new_full_X = tmp_full_X.merge(dem, left_on = ['state','communityname'],right_on = ['State_FIPS_Code','CHSI_County_Name'],how='left',suffixes=('_original', '_addition'))
new_full_X = new_full_X.replace('?', np.nan)
len(new_full_X)

1994

after the combination, we need to get the new label_y and remove some un-related attr

In [331]:
new_reg_y = new_full_X["ViolentCrimesPerPop"]
#deal with it to get the clf y
new_clf_y = new_reg_y.copy()
for index, item in enumerate(new_clf_y):
    if (item > 0.1):
        new_clf_y[index] = 1
    else:
        new_clf_y[index] = 0
len(list(new_full_X.columns[:]))

156

now remove the unrelated features

In [332]:
del new_full_X["state"]
del new_full_X["county"]
del new_full_X["community"]
del new_full_X["communityname"]
del new_full_X["fold"]
del new_full_X["ViolentCrimesPerPop"]
del new_full_X["County_FIPS_Code"]
del new_full_X["CHSI_State_Name"]
del new_full_X["CHSI_State_Abbr"]
del new_full_X["Strata_ID_Number"]
del new_full_X["CHSI_County_Name"]
del new_full_X["State_FIPS_Code"]
del new_full_X["US_ALE"]
del new_full_X["US_All_Death"]
del new_full_X["US_Health_Status"]
del new_full_X["US_Unhealthy_Days"]
len(list(new_full_X.columns[:]))

140

In [336]:
new_full_features = list(new_full_X.columns[:])
new_full_X = imp.fit_transform(new_full_X)

ii.Give accuracy, precision, and recall results with and without the new data.

In [338]:
#we using the dt to deal with this data
#init the DT clf
new_dt_clf = DecisionTreeClassifier()
new_ten_fold_accuracy = cross_val_score(new_dt_clf, new_full_X, new_clf_y, cv=10,scoring='accuracy')
new_ten_fold_precision = cross_val_score(new_dt_clf, new_full_X, new_clf_y, cv=10,scoring='precision')
new_ten_fold_recall = cross_val_score(new_dt_clf, new_full_X, new_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(new_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(new_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(new_ten_fold_recall)
print("the avg accuracy is:",sum(new_ten_fold_accuracy)/10)
print("the avg precision is:",sum(new_ten_fold_precision)/10)
print("the avg recall is:",sum(new_ten_fold_recall)/10)

the accuracy for 10-fold cv is:
[ 0.78109453  0.755       0.725       0.75879397  0.73366834  0.7638191
  0.81407035  0.75879397  0.73366834  0.73366834]
the precision for 10-fold cv is:
[ 0.84677419  0.796875    0.79508197  0.79844961  0.80172414  0.80508475
  0.828125    0.84070796  0.792       0.79365079]
the recall for 10-fold cv is:
[ 0.83333333  0.8         0.784       0.848       0.768       0.784       0.816
  0.792       0.752       0.784     ]
the avg accuracy is: 0.755757693942
the avg precision is: 0.809847341511
the avg recall is: 0.796133333333


In [341]:
#if without data
full_ten_fold_accuracy = cross_val_score(full_dt_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
full_ten_fold_precision = cross_val_score(full_dt_clf, full_X, full_clf_y, cv=10,scoring='precision')
full_ten_fold_recall = cross_val_score(full_dt_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10-fold cv is:")
print(full_ten_fold_accuracy)
print("the precision for 10-fold cv is:")
print(full_ten_fold_precision)
print("the recall for 10-fold cv is:")
print(full_ten_fold_recall)
print("the avg accuracy is:",sum(full_ten_fold_accuracy)/10)
print("the avg precision is:",sum(full_ten_fold_precision)/10)
print("the avg recall is:",sum(full_ten_fold_recall)/10)

the accuracy for 10-fold cv is:
[ 0.78606965  0.755       0.735       0.75376884  0.72864322  0.74371859
  0.8241206   0.75376884  0.73869347  0.73366834]
the precision for 10-fold cv is:
[ 0.85833333  0.8015873   0.76859504  0.82170543  0.79831933  0.8245614
  0.848       0.8245614   0.79661017  0.76923077]
the recall for 10-fold cv is:
[ 0.81746032  0.832       0.768       0.792       0.776       0.76        0.816
  0.792       0.752       0.8       ]
the avg accuracy is: 0.755245156129
the avg precision is: 0.811150417607
the avg recall is: 0.790546031746


we can find using the new data have not give a better result so far, maybe because the less of data

iii.Does the added data help? What features help? Does it matter what learning method you use?

from the above result we can find the data we added is not make so much help, only a little increase in accuracy

In [342]:
#check the main feature
new_dt_clf.fit(new_full_X, new_clf_y)
new_main_feature = dict(zip(new_full_features,new_dt_clf.feature_importances_))
for index,w in enumerate(sorted(new_main_feature, key=new_main_feature.get, reverse=True)):
    print(w ,new_main_feature[w])
    if index == 10:
        break

PctKids2Par 0.362863323984
racePctWhite 0.0916962793266
racePctHisp 0.0505171773405
PctLess9thGrade 0.0219382610508
PctEmplManu 0.0160142955179
MedRent 0.0159258528369
PctVacMore6Mos 0.0149708445158
PctSameState85 0.0145711031075
PctImmigRec10 0.0143954680319
PctHousOccup 0.0142232095289
HousVacant 0.013055380175


we can find that the important features did not changed a lot, all the features are from the original data

let's change the method to GNB

In [345]:
#init the gnb clf with data
new_nb_clf = GaussianNB()
new_nb_clf.fit(new_full_X, new_clf_y)
new_nb_clf_accuracy = cross_val_score(new_nb_clf, new_full_X, new_clf_y, cv=10,scoring='accuracy')
new_nb_clf_precision = cross_val_score(new_nb_clf, new_full_X, new_clf_y, cv=10,scoring='precision')
new_nb_clf_recall = cross_val_score(new_nb_clf, new_full_X, new_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(new_nb_clf_accuracy)
print("the precision for 10 cv is:")
print(new_nb_clf_precision)
print("the recall for 10 cv is:")
print(new_nb_clf_recall)
print("the avg accuracy is:",sum(new_nb_clf_accuracy)/10)
print("the avg precision is:",sum(new_nb_clf_precision)/10)
print("the avg recall is:",sum(new_nb_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.73134328  0.74        0.695       0.68844221  0.65829146  0.67839196
  0.72361809  0.71859296  0.69849246  0.63819095]
the precision for 10 cv is:
[ 0.93902439  0.92941176  0.94444444  0.92        0.88        0.87654321
  0.97297297  0.93670886  0.91139241  0.89552239]
the recall for 10 cv is:
[ 0.61111111  0.632       0.544       0.552       0.528       0.568       0.576
  0.592       0.576       0.48      ]
the avg accuracy is: 0.697036338408
the avg precision is: 0.920602043613
the avg recall is: 0.565911111111


In [346]:
#init the gnb clf without data
old_nb_clf = GaussianNB()
old_nb_clf.fit(full_X, full_clf_y)
old_nb_clf_accuracy = cross_val_score(old_nb_clf, full_X, full_clf_y, cv=10,scoring='accuracy')
old_nb_clf_precision = cross_val_score(old_nb_clf, full_X, full_clf_y, cv=10,scoring='precision')
old_nb_clf_recall = cross_val_score(old_nb_clf, full_X, full_clf_y, cv=10,scoring='recall')
print("the accuracy for 10 cv is:")
print(old_nb_clf_accuracy)
print("the precision for 10 cv is:")
print(old_nb_clf_precision)
print("the recall for 10 cv is:")
print(old_nb_clf_recall)
print("the avg accuracy is:",sum(old_nb_clf_accuracy)/10)
print("the avg precision is:",sum(old_nb_clf_precision)/10)
print("the avg recall is:",sum(old_nb_clf_recall)/10)

the accuracy for 10 cv is:
[ 0.73134328  0.73        0.705       0.71356784  0.67839196  0.69346734
  0.75376884  0.72361809  0.71859296  0.65829146]
the precision for 10 cv is:
[ 0.95        0.90804598  0.94594595  0.93589744  0.90666667  0.91025641
  0.975       0.94871795  0.91566265  0.90140845]
the recall for 10 cv is:
[ 0.6031746  0.632      0.56       0.584      0.544      0.568      0.624
  0.592      0.608      0.512    ]
the avg accuracy is: 0.710604177604
the avg precision is: 0.92976014858
the avg recall is: 0.582717460317


we can find with the data we have a better result.
there are two reasons to get this result:
1.our combine data number is too less that cannot influence the result.
2.the new attr that we import from the data.org are all unrelated attr.

we can test this by using the inner join to the data, then see what happens, we only use dt this time

In [348]:
little_new_full_X = tmp_full_X.merge(dem, left_on = ['state','communityname'],right_on = ['State_FIPS_Code','CHSI_County_Name'],how='inner',suffixes=('_original', '_addition'))
print("if we using the inner join, we can find the smaple number become:", len(little_new_full_X))
little_new_full_X = little_new_full_X.replace('?', np.nan)


if we using the inner join, we can find the smaple number become: 162


In [351]:
little_new_reg_y = little_new_full_X["ViolentCrimesPerPop"]
#deal with it to get the clf y
little_new_clf_y = little_new_reg_y.copy()
for index, item in enumerate(little_new_clf_y):
    if (item > 0.1):
        little_new_clf_y[index] = 1
    else:
        little_new_clf_y[index] = 0
del little_new_full_X["state"]
del little_new_full_X["county"]
del little_new_full_X["community"]
del little_new_full_X["communityname"]
del little_new_full_X["fold"]
del little_new_full_X["ViolentCrimesPerPop"]
del little_new_full_X["County_FIPS_Code"]
del little_new_full_X["CHSI_State_Name"]
del little_new_full_X["CHSI_State_Abbr"]
del little_new_full_X["Strata_ID_Number"]
del little_new_full_X["CHSI_County_Name"]
del little_new_full_X["State_FIPS_Code"]
del little_new_full_X["US_ALE"]
del little_new_full_X["US_All_Death"]
del little_new_full_X["US_Health_Status"]
del little_new_full_X["US_Unhealthy_Days"]
another_new_full_X = little_new_full_X.copy()
#another is the one without new data
del another_new_full_X["ALE"]
del another_new_full_X["Min_ALE"]
del another_new_full_X["Max_ALE"]
del another_new_full_X["All_Death"]
del another_new_full_X["Min_All_Death"]
del another_new_full_X["Max_All_Death"]
del another_new_full_X["CI_Min_All_Death"]
del another_new_full_X["CI_Max_All_Death"]
del another_new_full_X["Health_Status"]
del another_new_full_X["Min_Health_Status"]
del another_new_full_X["Max_Health_Status"]
del another_new_full_X["CI_Min_Health_Status"]
del another_new_full_X["CI_Max_Health_Status"]
del another_new_full_X["Unhealthy_Days"]
del another_new_full_X["Min_Unhealthy_Days"]
del another_new_full_X["Max_Unhealthy_Days"]
del another_new_full_X["CI_Min_Unhealthy_Days"]
del another_new_full_X["CI_Max_Unhealthy_Days"]
another_new_full_features = list(another_new_full_X.columns[:])

In [355]:
little_new_full_X = imp.fit_transform(little_new_full_X)
another_new_full_X = imp.fit_transform(another_new_full_X)

In [356]:
#using dt now with new feature
little_new_dt_clf = DecisionTreeClassifier()
little_new_ten_fold_accuracy = cross_val_score(little_new_dt_clf, little_new_full_X, little_new_clf_y, cv=10,scoring='accuracy')
little_new_ten_fold_precision = cross_val_score(little_new_dt_clf, little_new_full_X, little_new_clf_y, cv=10,scoring='precision')
little_new_ten_fold_recall = cross_val_score(little_new_dt_clf, little_new_full_X, little_new_clf_y, cv=10,scoring='recall')
print("the avg accuracy is:",sum(little_new_ten_fold_accuracy)/10)
print("the avg precision is:",sum(little_new_ten_fold_precision)/10)
print("the avg recall is:",sum(little_new_ten_fold_recall)/10)

the avg accuracy is: 0.753014705882
the avg precision is: 0.827144522145
the avg recall is: 0.834090909091


In [357]:
#using dt now without new feature
another_new_dt_clf = DecisionTreeClassifier()
another_new_ten_fold_accuracy = cross_val_score(another_new_dt_clf, another_new_full_X, little_new_clf_y, cv=10,scoring='accuracy')
another_new_ten_fold_precision = cross_val_score(another_new_dt_clf, another_new_full_X, little_new_clf_y, cv=10,scoring='precision')
another_new_ten_fold_recall = cross_val_score(another_new_dt_clf, another_new_full_X, little_new_clf_y, cv=10,scoring='recall')
print("the avg accuracy is:",sum(another_new_ten_fold_accuracy)/10)
print("the avg precision is:",sum(another_new_ten_fold_precision)/10)
print("the avg recall is:",sum(another_new_ten_fold_recall)/10)

the avg accuracy is: 0.758529411765
the avg precision is: 0.813268398268
the avg recall is: 0.868181818182


by the above analysis we can find add the new feature only increase the precision a little, so it means that the crime rate is not related to the health states in that county, which we added to predict the crime rate